In [14]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

import os
import os.path as path


batch_size = 128
num_classes = 10
epochs = 1

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], ' train samples')
print(x_test.shape[0], ' train samples')

# convert class vectors to binary class matrix
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

# building model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

# train model
model.fit(x_train, y_train, 
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy', score[1])


# export model
model_name = 'mnist_convnet'

file_path = 'out'
input_graph_file = model_name + '.pbtxt'
input_graph_file_path = file_path + '/' + input_graph_file
checkpoint_file = model_name + '.chkp'
checkpoint_file_path = file_path + '/' + checkpoint_file
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_frozen_graph_file_path = file_path + '/frozen_' + model_name + '.pb'
output_optimized_graph_file_path = file_path + '/optimized_' + model_name + '.pb'

input_node_names = "conv2d_1_input"
output_node_names = "dense_2/Softmax"

if not path.exists(file_path):
    os.mkdir(file_path)
    
# Save the graph
tf.train.write_graph(K.get_session().graph_def, file_path, input_graph_file)

# Save a checkpoint
saver = tf.train.Saver()
saver.save(K.get_session(), checkpoint_file_path)

# Freeze the graph. convert variables in the checkpoint file into Const Ops
freeze_graph.freeze_graph(input_graph_file_path, None, False, checkpoint_file_path,
                          output_node_name, restore_op_name, filename_tensor_name, 
                          output_frozen_graph_file_path, True, "")

# Optimize the model file
input_graph_def = tf.GraphDef()
with tf.gfile.Open(output_frozen_graph_file_path, "rb") as f:
    data = f.read()
    input_graph_def.ParseFromString(data)
    
output_graph_def = optimize_for_inference_lib.optimize_for_inference(
                    input_graph_def, 
                    [input_node_names], # an array of the input node(s)
                    [output_node_names], # an array of output nodes
                    tf.float32.as_datatype_enum)

# Save the optimized graph
with tf.gfile.FastGFile(output_optimized_graph_file_path, "wb") as f:
    f.write(output_graph_def.SerializeToString())
    
print ("graph saved!")

x_train shape: (60000, 28, 28, 1)
60000  train samples
10000  train samples
Train on 60000 samples, validate on 10000 samples
Epoch 1/1
60000/60000 [==============================] - 126s - loss: 0.3339 - acc: 0.8986 - val_loss: 0.0798 - val_acc: 0.9753
Test loss: 0.0797709787564
Test accuracy 0.9753
INFO:tensorflow:Restoring parameters from out/mnist_convnet.chkp


INFO:tensorflow:Restoring parameters from out/mnist_convnet.chkp


INFO:tensorflow:Froze 8 variables.


INFO:tensorflow:Froze 8 variables.


Converted 8 variables to const ops.
81 ops in the final graph.
graph saved!
